# Slicing and Views

ironArray is meant to store large arrays, but in practice you only want to access single elements or small parts of them. Here you will learn how to do that with the help of so called *views*. Views are just references to the part of a larger array that is interesting.

In order to see how slicing works in Iron Array, let's download some open data. In this case, we are going to work with precipitation data from a period of one month.

In [1]:
import zarr
import h5py
import xarray as xr
import numpy as np
import s3fs
import iarray as ia
from numcodecs import blosc, Blosc
import random
import os
import shutil

ia.set_config(nthreads=4, clevel=5, codec=ia.Codecs.LZ4, filters=[ia.Filters.SHUFFLE])
blosc.set_nthreads(ia.get_config().nthreads)

%load_ext memprofiler

In [2]:
def open_zarr(year, month, datestart, dateend):
    fs = s3fs.S3FileSystem(anon=True)

    datestring = 'era5-pds/zarr/{year}/{month:02d}/data/'.format(year=year, month=month)

    precip_zarr = xr.open_dataset(s3fs.S3Map(datestring + 'precipitation_amount_1hour_Accumulation.zarr/',
                                             s3=fs),
                                 engine="zarr")
    precip_zarr = precip_zarr.sel(time1=slice(np.datetime64(datestart), np.datetime64(dateend)))

    return precip_zarr.precipitation_amount_1hour_Accumulation

First of all, let's import the precipitation dataset into ironArray before proceeding with slicings:

In [3]:
if os.path.exists("precip_ia.iarray"):
    precip_ia = ia.load("precip_ia.iarray")
else:
    precip = open_zarr(1987, 10, '1987-10-01', '1987-10-30 23:59').data
    precip_ia = ia.numpy2iarray(precip)

print(precip_ia.info)
print(precip_ia.cratio)

type                : IArray
shape               : (720, 721, 1440)
chunkshape          : (256, 256, 512)
blockshape          : (32, 32, 64)
None
10.568012437007441


In this case, we have obtained a balanced chunkshape and blockshape (as we will see in [Optimizations Tips](#Optimizations-Tips), the chunkshape and the blockshape will be very important in slicings) since we have used the partition advice.

## Slicing Notation

In slicing tuples, Iron Array supports integers, `start:stop` slices (the `step` is not yet implemented) and Ellipsis `...`.

As in Python, all indices are zero-based: for the $i$-th index $n_i$, the valid range is $0 \le n_i < d_i$ where $d_i$ is the $i-th$ element of the shape of the array. Negative indices are interpreted as counting from the end of the array (i.e., if $n_i < 0$, it means $d_i + n_i$).

The simplest way to obtain a slice is using integers to acces to a value. In this case Iron Array will return a Python object.

In [4]:
s1 = precip_ia[5, 234, -55]
s1, type(s1)

(0.0, float)

As we said, Iron array also suports the `start:stop` notation. If `start` is not specified, the slice will start at the beginning of the array. In the same way, if `stop` is not specified, the slice will stop at the end of the array.

In [6]:
sl1 = precip_ia[34:555, 211:311, 300:]
sl2 = precip_ia[:, :-250, 300:500]
sl3 = precip_ia[:, :, :]
sl1, sl2, sl3

(<IArray (521, 100, 1140) np.float32>,
 <IArray (720, 471, 200) np.float32>,
 <IArray (720, 721, 1440) np.float32>)

Another interesting feature to use in slicing is the ellipsis object (`...`). This symbol expands the number of `:` objects to index all dimensions. There may only be a single ellipsis present.

In [7]:
sl1 = precip_ia[...]
sl2 = precip_ia[..., 5]
sl3 = precip_ia[500:100, ..., 400]
sl1, sl2, sl3

(<IArray (720, 721, 1440) np.float32>,
 <IArray (720, 721) np.float32>,
 <IArray (0, 721) np.float32>)

In ironArray the slicing tuples may be not completed at all (i.e. not all dimensions are indexed). If this is the case, Iron Array completes the remaining dimensions with `:`.

In [9]:
sl1 = precip_ia[:400, -500]
sl2 = precip_ia[5]
sl1, sl2

(<IArray (400, 1440) np.float32>, <IArray (721, 1440) np.float32>)

It should be noted that if an integer is used, the dimension of the matrix is reduced by one unit. If we want to keep the dimension, we can use a slice.

In [10]:
precip_ia[5].shape, precip_ia[5:6].shape

((721, 1440), (1, 721, 1440))

These two slices contains the same data. But the first dimension has been removed in the first slice since we have indexed it with an integer.

### Views

When a slice is performed in Iron Array, a view of the container is returned (like in numpy). You can always check whether an array is a view or not with the `is_view()` method:

In [11]:
%%time

s1 = precip_ia[2:300, 40:310, 500:1000]
precip_ia.is_view(), s1.is_view()

CPU times: user 322 µs, sys: 1 µs, total: 323 µs
Wall time: 327 µs


(False, True)

If we don't want a view, we can do a copy of the slice or get a numpy array using the `data` attribute:

In [12]:
%%time

s1 = precip_ia[2:300, 40:310, 500:1000].copy()
type(s1)

CPU times: user 1.42 s, sys: 408 ms, total: 1.83 s
Wall time: 2.16 s


iarray.iarray_container.IArray

In [13]:
%%time

s1 = precip_ia[2:300, 40:310, 500:1000].data
type(s1)

CPU times: user 379 ms, sys: 224 ms, total: 603 ms
Wall time: 424 ms


numpy.ndarray

So, retrieving the interesting data out of your IArray is pretty similar to NumPy convention.

At any rate, whenever you want to use the numpy advanced slicing features, you can always get a NumPy array out of an IArray (or a view of it) and apply your desired indexing there.  Remember that ironArray is meant for handling very large arrays, so there is no shame in getting the interesting slice as a NumPy object and then do your work over it.

Finally, indexing also applies to arrays that are stored persistently on disk.  ironArray will use the information about the data you want and will read and decompress only the part that is necessary.  And due to the double partitioning and fast compression codecs, this is in general very fast.

## Optimizations Tips

In this section we are going to fine-tune some of the Iron Array parameters to obtain a better performance. The first thing that we can modify is the chunkshape and the blockshape of the Array.

Lets suppose that we are going to slice the array always in the same dimension. For example, in this example we want to slice the array in the days dimension.

In [14]:
%%time
for i in range(precip_ia.shape[1]):
    _ = precip_ia[:, i, :].data

CPU times: user 57.3 s, sys: 8.15 s, total: 1min 5s
Wall time: 20.7 s


What happens if we optimize the chunkshape and the blockshape?

In [15]:
chunkshape = (precip_ia.shape[0], 8, precip_ia.shape[2])
blockshape = (64, 4, 64)

precip_ia_op = precip_ia.copy(chunkshape=chunkshape, blockshape=blockshape)

In [16]:
%%time
for i in range(precip_ia.shape[1]):
    _ = precip_ia_op[:, i, :].data

CPU times: user 11.9 s, sys: 1.64 s, total: 13.5 s
Wall time: 7.42 s


As can be seen, if we are always going to access in a specific dimension, it is very important to optimize the chunkshape and the blockshape of the array.

## Performance

Finally, we are going to perfom some benchmarks to compare Iron Array against zarr and hdf5 (on-disk scenario), another chunked and compressed array format. We will perform some random slices from each dimension of the array:

In [17]:
ind = random.sample(range(min(precip_ia.shape)), 10)

### In-memory performance

Let's start by measuring performance when data is in memory.

#### Zarr

In [18]:
compressor = Blosc(cname='lz4', clevel=5, shuffle=Blosc.SHUFFLE)
precip_zarr = zarr.array(precip_ia.data, chunks=chunkshape, compressor=compressor)

In [19]:
%%mprof_run zarr-dim0

for i in ind:
    zarr_0 = (precip_zarr[i, :, :])

memprofiler: used 6.86 MiB RAM (peak of 6.86 MiB) in 7.1496 s, total RAM usage 2734.75 MiB


In [20]:
%%mprof_run zarr-dim1

for i in ind:
    zarr_1 = precip_zarr[:, i, :]

memprofiler: used 4.01 MiB RAM (peak of 4.01 MiB) in 0.1951 s, total RAM usage 2739.03 MiB


In [21]:
%%mprof_run zarr-dim2

for i in ind:
    zarr_2 = precip_zarr[:, :, i]

memprofiler: used 0.08 MiB RAM (peak of 2.01 MiB) in 7.3814 s, total RAM usage 2739.13 MiB


#### Iron Array

In [22]:
%%mprof_run iarray-dim0

for i in ind:
    ia_0 = precip_ia_op[i, :, :].data

memprofiler: used 125.28 MiB RAM (peak of 125.28 MiB) in 0.6331 s, total RAM usage 2864.45 MiB


In [23]:
%%mprof_run iarray-dim1

for i in ind:
    ia_1 = precip_ia_op[:, i, :].data

memprofiler: used 27.34 MiB RAM (peak of 27.34 MiB) in 0.1532 s, total RAM usage 2891.80 MiB


In [24]:
%%mprof_run iarray-dim2

for i in ind:
    ia_2 = precip_ia_op[:, :, i].data

memprofiler: used -4.41 MiB RAM (peak of 3.98 MiB) in 0.9798 s, total RAM usage 2887.41 MiB


#### Results

In [25]:
np.testing.assert_almost_equal(ia_0, zarr_0)
np.testing.assert_almost_equal(ia_1, zarr_1)
np.testing.assert_almost_equal(ia_2, zarr_2)

Now, since we can not control the Python memory manager, we are only going to analyze the execution times:

In [26]:
%mprof_barplot -t "Slicing Performance (with an optimized dimension)" --variable time iarray-.* zarr-.* h5-.*

UsageError: Line magic function `%mprof_barplot` not found.


As we can see, in the optimized dimension the performance is very similar. However, in the other dimensions Iron Array overperformed by far zarr. This is due to the two level partitioning in the Iron Array arrays.

In this example, while zarr have to decompress all the chunks of the array in the non-optimized dimenisons, Iron Array only have to decompress the blocks that contains data from the slice.

### On-disk performance

Finally, we are going to run the same benchmark as before, but now the arrays will be on disk.

#### Zarr

In [40]:
%%time
zarr_urlpath = "slicing.zarr"

if not os.path.exists(zarr_urlpath):
    precip_zarr_disk = zarr.empty(precip_ia.shape, dtype=precip_ia.dtype,
                                  store=zarr_urlpath, chunks=chunkshape,
                                  compressor=compressor)
    precip_ia.copyto(precip_zarr_disk)


precip_zarr_disk = zarr.open(zarr_urlpath)

CPU times: user 1.22 ms, sys: 2.1 ms, total: 3.32 ms
Wall time: 3.43 ms


In [28]:
%%mprof_run zarr_disk-dim0

for i in ind:
    zarr_d_0 = precip_zarr_disk[i, :, :]

memprofiler: used -1.49 MiB RAM (peak of 16.34 MiB) in 10.5991 s, total RAM usage 830.41 MiB


In [29]:
%%mprof_run zarr_disk-dim1

for i in ind:
    zarr_d_1 = precip_zarr_disk[:, i, :]

memprofiler: used -35.93 MiB RAM (peak of 0.00 MiB) in 0.2816 s, total RAM usage 794.77 MiB


In [30]:
%%mprof_run zarr_disk-dim2

for i in ind:
    zarr_d_2 = precip_zarr_disk[:, :, i]

memprofiler: used 46.91 MiB RAM (peak of 62.87 MiB) in 10.2566 s, total RAM usage 841.70 MiB


#### HDF5

In [31]:
%%time
h5_urlpath = "slicing.hdf5"

if not os.path.exists(h5_urlpath):
    with h5py.File(h5_urlpath, "w") as f:
        h5_precip = f.create_dataset("h5_precip", precip_ia.shape, dtype=precip_ia.dtype, chunks=chunkshape, compression="lzf", shuffle=True)
        precip_ia.copyto(h5_precip)


h5_file = h5py.File(h5_urlpath, "r")
precip_h5_disk = h5_file['h5_precip']

In [32]:
%%mprof_run h5_disk-dim0

for i in ind:
    h5_d_0 = precip_h5_disk[i, :, :]

memprofiler: used -0.35 MiB RAM (peak of 3.68 MiB) in 81.0350 s, total RAM usage 872.36 MiB


In [33]:
%%mprof_run h5_disk-dim1

for i in ind:
    h5_d_1 = precip_h5_disk[:, i, :]

memprofiler: used -18.90 MiB RAM (peak of 0.04 MiB) in 2.0558 s, total RAM usage 845.35 MiB


In [34]:
%%mprof_run h5_disk-dim2

for i in ind:
    h5_d_2 = precip_h5_disk[:, :, i]

memprofiler: used 38.59 MiB RAM (peak of 51.14 MiB) in 84.4609 s, total RAM usage 883.96 MiB


In [35]:
h5_file.close()

#### Iron Array

In [41]:
%%time
ia_urlpath = "slicing.iarray"

if not os.path.exists(ia_urlpath):
    ia.save(ia_urlpath, precip_ia_op, chunkshape=chunkshape, blockshape=blockshape)  

precip_ia_op_disk = ia.open(ia_urlpath)

CPU times: user 870 µs, sys: 321 µs, total: 1.19 ms
Wall time: 1.16 ms


In [37]:
%%mprof_run iarray_disk-dim0

for i in ind:
    ia_d_0 = precip_ia_op_disk[i, :, :].data

memprofiler: used 3.79 MiB RAM (peak of 8.55 MiB) in 1.9372 s, total RAM usage 1175.34 MiB


In [38]:
%%mprof_run iarray_disk-dim1

for i in ind:
    ia_d_1 = precip_ia_op_disk[:, i, :].data

memprofiler: used 18.34 MiB RAM (peak of 18.34 MiB) in 0.3368 s, total RAM usage 1193.70 MiB


In [39]:
%%mprof_run iarray_disk-dim2

for i in ind:
    ia_d_2 = precip_ia_op_disk[:, :, i].data

memprofiler: used -31.94 MiB RAM (peak of 0.01 MiB) in 1.6363 s, total RAM usage 1161.77 MiB


#### Results

In [39]:
%mprof_barplot -t "Slicing Performance on disk (with an optimized dimension)" --variable time iarray_disk-.* zarr_disk-.* h5_disk-.*

As we can see, the results that go through the optimized dimension are a bit better in zarr than ironArray. However, as for the results across the other dimensions, ironArray is faster than zarr, although not as good as in the in-memory version.  HDF5 in that case is not competitive in any dimension.

To conclude, by providing two levels of partitioning, ironArray has more fine-grained flexibility in adapting to different I/O patterns.  Also, this two-level partitions allow for better reducing the number of data read from disk (or memory) than similar solutions with just one level partitioning.